In [ ]:
%%capture
!pip install langchain langchain_openai langchain_google_genai langchain_core langchain_community
!pip install wikipedia chromadb
!pip install pyowm
!pip install langchainhub
!pip install newspaper3k
!pip install newsapi-python
!pip install pycountry
!pip install google-search-results

In [ ]:
openaiKey = "*****************************"
googleKey = "********************************"

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = openaiKey
os.environ['GOOGLE_API_KEY'] = googleKey

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate  , ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.chains import LLMChain

from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

from langchain.chains.router import MultiRouteChain
from langchain.chains.router.llm_router import LLMRouterChain , RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

from langchain_community.document_loaders import WikipediaLoader

from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.runnables import RunnablePassthrough

from langchain_community.document_loaders import WeatherDataLoader
from langchain.tools import BaseTool, StructuredTool, tool

from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_community.document_loaders import NewsURLLoader
from newsapi import NewsApiClient

In [ ]:
llm1 = ChatOpenAI(model_name ='gpt-3.5-turbo' , temperature = 0)
llm2 = ChatGoogleGenerativeAI(model = 'gemini-pro' , temperature = 0 , convert_system_message_to_human=True)

##1

### **1.1 Compare the responses of two LLMs**

In [ ]:
def CompareLLM(question):
  quest = "Define What is Machine Learning in 3 lines"
  prompt = PromptTemplate(input_variables=[],template="")

  question_prompt = PromptTemplate.from_template("{question}")
  complete_chain_prompt = PromptTemplate.from_template("""
  Give me the comparision scores out of 100 for each resposne aganist to the given question:
  question:{question}
  response 1:{response1}
  response 2:{response2}
  Also suggest which response is best.
  """)

  first_chain = question_prompt | llm1 | StrOutputParser()
  second_chain = question_prompt | llm2 | StrOutputParser()
  third_chain = complete_chain_prompt | llm1 | StrOutputParser()

  complete_chain = ({
          "question": itemgetter("question"),
          "response1": first_chain ,
          "response2" : second_chain ,
          }
      | RunnablePassthrough.assign(final=third_chain)
      )

  final_res = complete_chain.invoke({"question":quest})

  for i in final_res:
    print(f"{i} : \n{final_res[i]}")
    print("\n")

CompareLLM("Define What is Machine Learning in 3 lines")

question : 
Define What is Machine Learning in 3 lines


response1 : 
Machine learning is a subset of artificial intelligence that allows computers to learn and improve from experience without being explicitly programmed. It involves algorithms that analyze data, identify patterns, and make decisions or predictions based on that data. Machine learning is used in a wide range of applications, from recommendation systems to self-driving cars.


response2 : 
- Machine learning is a subfield of artificial intelligence that gives computers the ability to learn without being explicitly programmed.
- Machine learning algorithms use historical data as input to predict new output values.
- The goal of machine learning is to create algorithms that can generalize from past experiences to make accurate predictions about the future.


final : 
Response 1:
- Accuracy: 95/100
- Conciseness: 90/100
- Relevance: 100/100

Response 2:
- Accuracy: 90/100
- Conciseness: 95/100
- Relevance: 95/100

Based on

### **1.2 MutiRouteChain for facts and creativity**

In [ ]:
def MultiRouteExecutor(query):
  facts = """
  You are good at past historical data or facts .
  So , Answer the following question:
  question : {input}
  """

  creative = """
  You are good at creating content for the following subject:
  subject : {input}
  """

  prompt_infos = [
    {
        "name":"fact" ,
        "description":"Good for answering questions about historical data" ,
        "prompt_template": facts
    }  ,
    {
        "name":"creative" ,
        "description":"Good for generating content about given subject" ,
        "prompt_template":creative
    }
  ]


  destination_chain = {}

  for prompt_info in prompt_infos:
    name = prompt_info['name']
    template = prompt_info['prompt_template']
    chat_prompt = ChatPromptTemplate.from_template(template  = template)
    chain = LLMChain(llm=llm2 , prompt = chat_prompt)
    destination_chain[name] = chain

  default_prompt = ChatPromptTemplate.from_template(template = "{input}")
  default_chain = LLMChain(llm=llm2 , prompt = default_prompt)

  destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
  destination_str = "\n".join(destinations)

  router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)
  router_prompt = PromptTemplate(template=router_template,
                                input_variables=["input"],
                                output_parser=RouterOutputParser())
  router_chain = LLMRouterChain.from_llm(llm=llm2, prompt=router_prompt)
  chain = MultiRouteChain(router_chain = router_chain , destination_chains = destination_chain , default_chain=default_chain , verbose=True )

  result = chain.invoke(query)

  return result['text']


print(MultiRouteExecutor("Best places to visit in India."))
print(MultiRouteExecutor("Tell me some thing about nature ."))



> Entering new MultiRouteChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


fact: {'input': 'Best places to visit in India.'}
> Finished chain.
1. **Taj Mahal, Agra:** This iconic mausoleum is a UNESCO World Heritage Site and one of the most recognizable buildings in the world. It was built by Mughal emperor Shah Jahan in memory of his wife, Mumtaz Mahal.

2. **Red Fort, Delhi:** This historic fort was built by the Mughal emperor Shah Jahan in the 17th century. It is a UNESCO World Heritage Site and a popular tourist destination.

3. **Qutub Minar, Delhi:** This towering minaret is the tallest brick minaret in the world. It was built by the Delhi Sultanate in the 13th century.

4. **Gateway of India, Mumbai:** This iconic archway was built by the British in 1924 to commemorate the visit of King George V and Queen Mary. It is a popular tourist destination and a symbol of Mumbai.

5. **Victoria Memorial, Kolkata:** This grand memorial was built in the early 20th century to commemorate the reign of Queen Victoria. It is a UNESCO World Heritage Site and a popular 

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


fact: {'input': 'Tell me some thing about nature .'}
> Finished chain.
1. **Biodiversity:** Nature is incredibly diverse, with an estimated 8.7 million species on Earth, each with unique adaptations and ecological roles. This diversity is essential for the functioning of ecosystems and the survival of life on Earth.

2. **Interconnectedness:** All living things in nature are interconnected and interdependent. Changes in one part of an ecosystem can have ripple effects throughout the entire system. For example, the decline of a single species can lead to the decline of other species that rely on it for food or habitat.

3. **Resilience:** Nature has a remarkable ability to recover from disturbances, such as fires, floods, and droughts. This resilience is due to the complex interactions between species and their environment. However, human activities, such as pollution and climate change, can overwhelm nature's ability to recover.

4. **Beauty:** Nature is often seen as beautiful and awe

##2

### **2.1 Retrieve most similar documents from vector store**

In [ ]:


docs = WikipediaLoader(query="HUNTER X HUNTER", load_max_docs=5).load()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(docs)

# create the open-source embedding function
embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "Tell me all about HUNTER X HUNTER"
final_docs = db.similarity_search(query , k=2)

final_docs_str = "\n".join([page.page_content for page in final_docs])

prompt = PromptTemplate.from_template("""
Give me the answer for the following question using the context
question : {question}
context  : {context}
""")

chain = prompt | llm2 | StrOutputParser()
question = "How many number of copies are released for Hunter × Hunter?"
ans = chain.invoke({"question":question,"context":final_docs_str})

print(ans)

84 million


### **2.2 Know the weather of the day**

In [ ]:
loader = WeatherDataLoader.from_params(
    ["chennai", "vellore"], openweathermap_api_key="0e3c31016f43cc752004df9994198a40"
)

documents = loader.load()

documents_str = "\n".join([page.page_content for page in documents])

prompt = ChatPromptTemplate.from_template("""
Answer the following question using the given context:
question : {question}
context : {context}
""")

chain = prompt | llm2 | StrOutputParser()

res = chain.invoke({"question":"What is the humidity in chennai" , "context":documents_str})

print(res)

66%


## **3 . Use Multi chain to get the dish recipes based on seasonality**

In [ ]:
def Dish_Recipe(seasonality):
  course_prompt = ChatPromptTemplate.from_template(""""
  Give me a 3-course vegetarian meal that uses seasonal ingredients in 3 lines:
  seasonality : {seasonality}
  """)
  course_chain = course_prompt | llm2 | StrOutputParser()
  course_res = course_chain.invoke({"seasonality":seasonality})

  print("######################  Course  ############################")
  print(course_res)

  dish_prompt = ChatPromptTemplate.from_template(""""
  Give me a dish for following courses in 3 lines:
  courses : {courses}
  """)
  dish_chain = dish_prompt | llm2 | StrOutputParser()
  dish_res = dish_chain.invoke({"courses":course_res})

  print("#######################  Dish  ###########################")
  print(dish_res)

  recipe_prompt = ChatPromptTemplate.from_template(""""
  Give me a recipe for following dishes:
  dish : {dish}
  """)
  recipe_chain = recipe_prompt | llm2 | StrOutputParser()

  recipe_res = recipe_chain.invoke({"dish":dish_res})

  print("#####################  Recipe  #############################")
  print(recipe_res)

Dish_Recipe("Winter")

######################  Course  ############################
1. Roasted Butternut Squash Soup with Sage and Thyme Croutons
2. Grilled Portobello Mushrooms with Balsamic Glaze and Roasted Vegetables
3. Apple and Cranberry Crumble with Vanilla Bean Ice Cream
#######################  Dish  ###########################
1. Roasted Butternut Squash Soup: A creamy and flavorful soup made with roasted butternut squash, sage, and thyme croutons.
2. Grilled Portobello Mushrooms: Portobello mushrooms grilled to perfection and topped with a balsamic glaze and roasted vegetables.
3. Apple and Cranberry Crumble: A classic dessert featuring a sweet and tart filling of apples and cranberries topped with a buttery crumble and served with vanilla bean ice cream.
#####################  Recipe  #############################
**Roasted Butternut Squash Soup:**

**Ingredients:**

* 1 medium butternut squash, peeled and cubed
* 1 tablespoon olive oil
* 1/2 teaspoon salt
* 1/4 teaspoon black pepper
* 1/2 teaspo

##4

### **4.1 Use agents to convert given content to another style of format( required format)**

In [ ]:
passage = """
The sun was setting, casting a warm glow over the tranquil lake. The gentle breeze whispered through the trees,
creating a symphony of rustling leaves. It was a serene evening, perfect for quiet reflection.
"""
template = """
Examine the stylistic elements such as tone, language choice, punctuation, sentence structure,
and overall formality or informality in the following passage:
'{passage}'
Analyze how these elements contribute to the overall style and atmosphere of the passage.
"""
style_analysis_prompt =  ChatPromptTemplate.from_template(template = template)

text_style_analysis_chain = style_analysis_prompt | llm2 | StrOutputParser()

res_style_analysis = text_style_analysis_chain.invoke({"passage":passage})

target_format = """
Blog post format
"""

transformation_template = """
Analyse the style of the following passage and convert the style of the passage into target format:
passage : {passage}
target format : {target format}
"""



@tool
def StyleTransfer(passage, target_format):
    """Style transfer for the given passage into target format."""
    transformation_template = """
    Analyse the style of the following passage and convert the style of the passage into target format:
    passage : {passage}
    target format : {target format}
    """
    transformation_prompt = ChatPromptTemplate.from_template(template = transformation_template)
    transformation_chain = transformation_prompt | llm2 | StrOutputParser()
    res = transformation_chain.invoke({"passage":passage , "target format":target_format})
    return res

tools = [StyleTransfer]


prompt = ChatPromptTemplate.from_messages(
    [

        ("user", "Convert the given passage into target format:\n passage: {passage} \n target format : {target format} "),    # Give me the {passage} into {target format} style of content
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm1.bind_tools(tools)

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "passage": lambda x: x["passage"],
        "target format": lambda x: x["target format"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

style_transformed = agent_executor.invoke({"passage":passage , "target format":target_format})



> Entering new AgentExecutor chain...

Invoking: `StyleTransfer` with `{'passage': 'The sun was setting, casting a warm glow over the tranquil lake. The gentle breeze whispered through the trees, creating a symphony of rustling leaves. It was a serene evening, perfect for quiet reflection.', 'target_format': 'Blog post format'}`


**Title: A Tranquil Evening by the Lake**

**Introduction:**

As the sun gracefully descends, casting a warm and inviting glow upon the tranquil lake, a sense of serenity washes over me. The gentle breeze, like a whisper from nature, rustles through the trees, creating a symphony of rustling leaves. It is an evening of tranquility, perfect for quiet reflection and peaceful contemplation.

**Body:**

The lake, like a mirror, reflects the fading hues of the sky, painting a breathtaking canvas of colors. The water's surface is still and undisturbed, save for the occasional ripple caused by a leaping fish or a passing water strider. The air is filled with the s

In [ ]:
print(style_transformed['output'])

The given passage has been converted into a blog post format. Here is the transformed content:

---

**Title: A Tranquil Evening by the Lake**

**Introduction:**

As the sun gracefully descends, casting a warm and inviting glow upon the tranquil lake, a sense of serenity washes over me. The gentle breeze, like a whisper from nature, rustles through the trees, creating a symphony of rustling leaves. It is an evening of tranquility, perfect for quiet reflection and peaceful contemplation.

**Body:**

The lake, like a mirror, reflects the fading hues of the sky, painting a breathtaking canvas of colors. The water's surface is still and undisturbed, save for the occasional ripple caused by a leaping fish or a passing water strider. The air is filled with the sweet fragrance of wildflowers, carried by the gentle breeze.

As I sit on the shore, lost in the beauty of the moment, I feel a sense of peace and tranquility wash over me. The worries and stresses of the day seem to melt away, replac

### **4.2 Use agents to summarise the required or relevant articles**

In [ ]:
news_api_key = "2ace72c9c7a04dc889bd75be7973f8e9"

In [ ]:

newsapi = NewsApiClient(api_key=news_api_key)

all_articles = newsapi.get_everything(q='technology',
                                      language='en',
                                      sort_by='popularity',
                                      page=5)

urls = [all_articles['articles'][i]['url'] for i in range(0,5)]

loader = NewsURLLoader(urls=urls)
data = loader.load()

content = "\n".join([data[i].page_content for i in range(len(data))])

article_template = """
Summarise the following content:
content : {article content}
"""
article_prompt = ChatPromptTemplate.from_template(article_template)
article_chain = article_prompt | llm1 | StrOutputParser()

article_res = article_chain.invoke({"article content":content})

In [ ]:
print(article_res)

The content discusses the transformative effects of GLP-1 medications, such as Mounjaro, on weight loss and behavior. The author shares their personal experience with these medications, highlighting how they have redefined their relationship with food, shame, and self-perception. The article delves into the broader implications of GLP-1s on issues of free will, addiction, criminal justice, and childhood obesity. The author reflects on the profound impact of these medications on their life and the complex ethical questions they raise.
